In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
df = pd.read_csv("D:/winequality.csv")

In [5]:
X = df.drop('quality', axis=1).values
y = df['quality'].values
y = (y >= 6).astype(int)

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [11]:
def build_tf_model(input_shape):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=input_shape))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
tf_model = build_tf_model(X_train.shape[1])

C:\Users\indum\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)  
        self.fc2 = nn.Linear(32, 1)  

    def forward(self, x):
        x = torch.relu(self.fc1(x))  
        x = torch.sigmoid(self.fc2(x))  
        return x

In [15]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
pytorch_model = SimpleNN(X_train.shape[1])

In [17]:
start_time_tf = time.time()
tf_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
end_time_tf = time.time()
train_time_tf = end_time_tf - start_time_tf
print(f"TensorFlow Training Time: {train_time_tf:.4f} seconds")

TensorFlow Training Time: 3.6443 seconds


In [19]:
start_time_pt = time.time()
criterion = nn.BCELoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.001)
for epoch in range(10):
    outputs = pytorch_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

end_time_pt = time.time()
train_time_pt = end_time_pt - start_time_pt
print(f"PyTorch Training Time: {train_time_pt:.4f} seconds")

PyTorch Training Time: 4.5138 seconds


In [21]:
y_pred_tf = tf_model.predict(X_test)
y_pred_tf = (y_pred_tf > 0.5).astype(int)
accuracy_tf = accuracy_score(y_test, y_pred_tf)
print(f"TensorFlow Test Accuracy: {accuracy_tf:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
TensorFlow Test Accuracy: 0.7500


In [23]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
pytorch_model.eval()
with torch.no_grad():
    y_pred_pt = pytorch_model(X_test_tensor)
    y_pred_pt = (y_pred_pt > 0.5).float()
    
accuracy_pt = accuracy_score(y_test, y_pred_pt)
print(f"PyTorch Test Accuracy: {accuracy_pt:.4f}")

PyTorch Test Accuracy: 0.6062


In [25]:
comparison_df = pd.DataFrame({
    'Framework': ['TensorFlow', 'PyTorch'],
    'Training Time (s)': [train_time_tf, train_time_pt],
    'Test Accuracy': [accuracy_tf, accuracy_pt]
})

print("\nFramework Comparison:")
print(comparison_df)


Framework Comparison:
    Framework  Training Time (s)  Test Accuracy
0  TensorFlow           3.644334        0.75000
1     PyTorch           4.513783        0.60625
